# QDD, Feedback Master equation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt
import cmath
import statsmodels.api as sm
import scipy as sp
%run Functions.ipynb

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

### State vectors and field operators

Definition of the states corresponding to the two potential wells:

In [2]:
psi_0 = qt.fock(2,0)
psi_1 = qt.fock(2,1)
O_I = qt.tensor(psi_0,psi_1)
I_O = qt.tensor(psi_1,psi_0)

Initial conditions:

In [3]:
initial_ket = +np.cos(np.pi/8)*I_O+1j*np.sin(np.pi/8)*O_I
initial_ket.dag()*initial_ket

Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[1.]]

Definition of the field operators:

In [4]:
a_1 = qt.tensor(qt.destroy(2), qt.qeye(2))
a_2 = qt.tensor(qt.qeye(2), qt.destroy(2))

Definition of the number operators:

In [5]:
N_1 = a_1.dag()*a_1
N_2 = a_2.dag()*a_2

Bloch operators:

In [6]:
x = a_1.dag()*a_2+a_2.dag()*a_1
y = 1j*(-a_1.dag()*a_2+a_2.dag()*a_1)
z = a_1.dag()*a_1-a_2.dag()*a_2

Expectation values initial conditions:

In [7]:
print('expected initial x', qt.expect(x, initial_ket))
print('expected initial y', qt.expect(y, initial_ket))
print('expected initial z', qt.expect(z, initial_ket))

expected initial x 0.0
expected initial y 0.7071067811865476
expected initial z 0.7071067811865475


Initial density operator:

In [8]:
rho_0 = initial_ket*initial_ket.dag()

In [9]:
rho_0

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.        +0.j         0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.14644661+0.j         0.        +0.35355339j
  0.        +0.j        ]
 [0.        +0.j         0.        -0.35355339j 0.85355339+0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.        +0.j        ]]

### Feedback Master Equation

Parameters of the feedback ME:

In [10]:
omega_1 = 0 
omega_2 = 0
Omega = 0.3 #Exchange energy
mu = 0.02 #Feedback strenght
T = 10. #Tunneling coefficient
chi = 0.1 #Tunneling coefficient variation
gamma = abs(chi)/abs(T)
dt = 0.1
times = np.arange(0, 30, dt)
NUMBER_OF_TRAJECTORIES = 20000

Operators:

In [11]:
F = mu*x #feedback operator 
A = T + chi*N_1 
Jump =A-1j*F#Jump operator for the ME in Lindblad form
H = Omega*(a_1.dag()*a_2 + a_2.dag()*a_1)+1/2*(A*F+F*A)-T*F #Hamiltonian

In [12]:
#noise order
print('noise order', 1/np.sqrt(dt))

noise order 3.162277660168379


Current operators:

In [13]:
I = abs(T)*(1 + 2*gamma*np.cos(cmath.phase(T) - cmath.phase(chi))*(a_1.dag()*a_1 - a_2.dag()*a_2))
I_1 = chi*z

In [14]:
I_1

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.   0.   0.   0. ]
 [ 0.  -0.1  0.   0. ]
 [ 0.   0.   0.1  0. ]
 [ 0.   0.   0.   0. ]]

### Simulation

In [15]:
stoc_solution = qt.smesolve(H, rho_0, times,
                            c_ops=[],
                            sc_ops=[Jump],
                            m_ops = [I_1],
                            ntraj=NUMBER_OF_TRAJECTORIES,
                            nsubsteps=100,
                            store_measurement=True,
                            dW_factors=[1.])

10.0%. Run time: 748.02s. Est. time left: 00:01:52:12
20.0%. Run time: 1507.16s. Est. time left: 00:01:40:28
30.0%. Run time: 2261.92s. Est. time left: 00:01:27:57
40.0%. Run time: 2960.37s. Est. time left: 00:01:14:00
50.0%. Run time: 3681.12s. Est. time left: 00:01:01:21
60.0%. Run time: 62176.34s. Est. time left: 00:11:30:50
70.0%. Run time: 62755.76s. Est. time left: 00:07:28:15


KeyboardInterrupt: 

### Unconditional solution

In [ ]:
fig, ax = plt.subplots()
ax.set_title(f'Unconditional solution for {NUMBER_OF_TRAJECTORIES} trajectories')
ax.plot(times, np.array(stoc_solution.measurement).mean(axis=0)[:],
        'r', label=r'$J_x$')
#ax.set_xlim(0,10)

### Example of conditional solution

In [ ]:
fig, ax = plt.subplots()
ax.set_title('Stochastic Master Equation - double dot, single trajectory')
ax.plot(times,stoc_solution.measurement[5],
        'r', lw=2, label=r'$J_x$')

### Save

In [ ]:
Ens = reshape_measurement(NUMBER_OF_TRAJECTORIES, len(times), stoc_solution.measurement)

In [ ]:
np.savetxt('TrajectoriesFeedback2/feed_best_20.txt', Ens)